# 1 Pakete importieren

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from stargazer.stargazer import Stargazer
import statsmodels.formula.api as smf 
import statsmodels.api as sm
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from scipy.stats import linregress
from scipy import stats

# 2 CSV-Dateien einlesen

In [8]:
csv_1 = "data/kunden.csv"
csv_2 = "data/besucher.csv"

In [9]:
df_1 = pd.read_csv(csv_1, sep = ";")
df_2 = pd.read_csv(csv_2, sep = ";")

# 3 Überblick über die Datentypen verschaffen

In [10]:
df_1.dtypes

Alter           int64
Einkommen     float64
Preis         float64
Geschlecht    float64
Zeit          float64
KundeNr        object
dtype: object

In [11]:
df_2.dtypes

Alter          int64
Einkommen     object
Geschlecht     int64
Zeit          object
KundeNr       object
dtype: object

Zu ändern sind später: Einkommen --> float; Geschlecht --> int; Zeit --> float

# 4.1 df_1 Geschlecht

In [12]:
df_1.isnull().sum()

Alter         0
Einkommen     0
Preis         0
Geschlecht    5
Zeit          0
KundeNr       0
dtype: int64

In [13]:
df_1["Geschlecht"] = df_1["Geschlecht"].fillna(1) # Für 1 (Männlich) entschieden, da die meisten Ausprägungen 1 waren
df_1.isna().sum()

Alter         0
Einkommen     0
Preis         0
Geschlecht    0
Zeit          0
KundeNr       0
dtype: int64

# df_1 Einkommen

In [14]:
df_1["Einkommen"].mean()

964474.8175955408

In [15]:
median_einkommen = df_1["Einkommen"].median()
median_einkommen

59013.0

In [16]:
df_1["Einkommen"] = df_1["Einkommen"].apply(lambda x: median_einkommen if x <= 0 or x >= 1000000 else x)

In [17]:
df_1.describe()

,Alter,Einkommen,Preis,Geschlecht,Zeit
count,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000
mean,50.481884,58894.998188,88975.198714,0.639493,44.375906
std,12.225059,16509.072600,24464.067781,0.480365,10.848040
min,30.000000,14547.000000,21471.650000,0.000000,12.000000
25%,40.000000,46697.000000,70609.422500,0.000000,37.000000
50%,51.000000,59013.000000,88931.455000,1.000000,44.000000
75%,61.000000,70281.750000,107093.375000,1.000000,52.000000
max,120.000000,107689.000000,165482.310000,1.000000,80.000000


# df_1 Alter 

df_1["Alter"].value_counts()

In [19]:
modalwert_alter = df_1["Alter"].mode()[0]
modalwert_alter

55

In [21]:
anzahl_personen_ueber_70 = len(df_1[df_1["Alter"] > 70])
anzahl_personen_ueber_70

2

In [22]:
df_1["Alter"] = df_1["Alter"].apply(lambda x: modalwert_alter if x >= 100 else x)

In [23]:
df_1.describe()

,Alter,Einkommen,Preis,Geschlecht,Zeit
count,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000
mean,50.382246,58894.998188,88975.198714,0.639493,44.375906
std,11.953017,16509.072600,24464.067781,0.480365,10.848040
min,30.000000,14547.000000,21471.650000,0.000000,12.000000
25%,40.000000,46697.000000,70609.422500,0.000000,37.000000
50%,51.000000,59013.000000,88931.455000,1.000000,44.000000
75%,61.000000,70281.750000,107093.375000,1.000000,52.000000
max,70.000000,107689.000000,165482.310000,1.000000,80.000000


# 4.2 df_1 Daten umformen

In [24]:
df_1["Geschlecht"] = df_1["Geschlecht"].astype("int64")

In [25]:
df_1.dtypes

Alter           int64
Einkommen     float64
Preis         float64
Geschlecht      int64
Zeit          float64
KundeNr        object
dtype: object

# 5  df_2 Daten umformen

In [26]:
df_2["Einkommen"] = pd.to_numeric(df_2["Einkommen"].str.replace(',', '.'), errors="coerce")

In [ ]:
df_2["Zeit"] = pd.to_numeric(df_2["Zeit"].str.replace(',', '.'), errors="coerce")

In [29]:
df_2.dtypes

Alter           int64
Einkommen     float64
Geschlecht      int64
Zeit          float64
KundeNr        object
dtype: object

In [30]:
df_2.describe()

,Alter,Einkommen,Geschlecht,Zeit
count,891.000000,891.000000,891.000000,891.000000
mean,49.881033,59367.936188,0.578002,35.166105
std,11.865889,16164.358221,0.494156,10.476802
min,30.000000,10802.033307,0.000000,3.000000
25%,40.000000,47816.611417,0.000000,28.000000
50%,49.000000,59163.823681,1.000000,35.000000
75%,60.000000,71001.378839,1.000000,43.000000
max,70.000000,108569.087021,1.000000,68.000000


# 6 Zusammenfügen von df_1 und df_2 zu df_3

In [31]:
df_3 = pd.concat([df_1, df_2],
                axis = 0, # Über die Zeilen zusammenfügen
                ignore_index = True)

In [32]:
df_3

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.000000,88160.31,1,43.0,K0310
1,54,77644.000000,103145.70,1,40.0,K1042
2,55,44341.000000,80565.16,0,37.0,K0382
3,49,67271.000000,83949.89,0,42.0,K0498
4,46,49832.000000,93781.58,0,41.0,K0552
...,...,...,...,...,...,...
1990,32,34391.824965,NaN,1,48.0,K1211
1991,41,68516.495439,NaN,0,34.0,K1375
1992,65,89685.333441,NaN,0,23.0,K1965
1993,68,95714.305268,NaN,0,32.0,K1540


# 7 Einkommen runden

In [33]:
df_3["Einkommen"] = df_3["Einkommen"].round(2)
df_3

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.00,88160.31,1,43.0,K0310
1,54,77644.00,103145.70,1,40.0,K1042
2,55,44341.00,80565.16,0,37.0,K0382
3,49,67271.00,83949.89,0,42.0,K0498
4,46,49832.00,93781.58,0,41.0,K0552
...,...,...,...,...,...,...
1990,32,34391.82,NaN,1,48.0,K1211
1991,41,68516.50,NaN,0,34.0,K1375
1992,65,89685.33,NaN,0,23.0,K1965
1993,68,95714.31,NaN,0,32.0,K1540


# 8.1 Textdatei einlesen

In [38]:
text_datei = "data/geo.txt"
df_text = pd.read_csv(text_datei, header=None, names=["Platzhalter"])
df_text

,Platzhalter
0,KundeNr\tNiederlassung
1,K0001\tSachsen
2,K0002\tNRW
3,K0003\tHessen
4,K0004\tBayern
...,...
1991,K1991\tHessen
1992,K1992\tBaden-Württemberg
1993,K1993\tNordrhein-Westfalen
1994,K1994\tBerlin


# 8.2 Überschriften 

In [39]:
df_text[["KundeNR", "Niederlassung"]] = df_text["Platzhalter"].str.split(expand=True)
df_text

,Platzhalter,KundeNR,Niederlassung
0,KundeNr\tNiederlassung,KundeNr,Niederlassung
1,K0001\tSachsen,K0001,Sachsen
2,K0002\tNRW,K0002,NRW
3,K0003\tHessen,K0003,Hessen
4,K0004\tBayern,K0004,Bayern
...,...,...,...
1991,K1991\tHessen,K1991,Hessen
1992,K1992\tBaden-Württemberg,K1992,Baden-Württemberg
1993,K1993\tNordrhein-Westfalen,K1993,Nordrhein-Westfalen
1994,K1994\tBerlin,K1994,Berlin


In [40]:
df_text.columns

Index(['Platzhalter', 'KundeNR', 'Niederlassung'], dtype='object')

In [41]:
df_text = df_text.drop("Platzhalter", axis=1)

In [42]:
df_text.columns

Index(['KundeNR', 'Niederlassung'], dtype='object')

# 9 Im df_3 neue Spalte erstellen

In [43]:
df_3 = df_3.copy()
df_3["Niederlassung"] = None
df_3

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Niederlassung
0,64,66894.00,88160.31,1,43.0,K0310,None
1,54,77644.00,103145.70,1,40.0,K1042,None
2,55,44341.00,80565.16,0,37.0,K0382,None
3,49,67271.00,83949.89,0,42.0,K0498,None
4,46,49832.00,93781.58,0,41.0,K0552,None
...,...,...,...,...,...,...,...
1990,32,34391.82,NaN,1,48.0,K1211,None
1991,41,68516.50,NaN,0,34.0,K1375,None
1992,65,89685.33,NaN,0,23.0,K1965,None
1993,68,95714.31,NaN,0,32.0,K1540,None


# 10 Spalte umbenennen 

In [44]:
df_3 = df_3.rename(columns={"KundeNr": "KundeNR"})
df_3.dtypes

Alter              int64
Einkommen        float64
Preis            float64
Geschlecht         int64
Zeit             float64
KundeNR           object
Niederlassung     object
dtype: object

# 11 Text-Datei mit CSV mergen

In [45]:
merged_df = pd.merge(df_3, df_text[["KundeNR", "Niederlassung"]], on="KundeNR", how="left")
merged_df

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNR,Niederlassung_x,Niederlassung_y
0,64,66894.00,88160.31,1,43.0,K0310,None,Bayern
1,54,77644.00,103145.70,1,40.0,K1042,None,Berlin
2,55,44341.00,80565.16,0,37.0,K0382,None,Baden-Württemberg
3,49,67271.00,83949.89,0,42.0,K0498,None,Bayern
4,46,49832.00,93781.58,0,41.0,K0552,None,Hessen
...,...,...,...,...,...,...,...,...
1990,32,34391.82,NaN,1,48.0,K1211,None,Hessen
1991,41,68516.50,NaN,0,34.0,K1375,None,Sachsen
1992,65,89685.33,NaN,0,23.0,K1965,None,Bayern
1993,68,95714.31,NaN,0,32.0,K1540,None,Bayern


# 12 Spalte umbennen und löschen

In [47]:
merged_df = merged_df.rename(columns={"Niederlassung_y": "Niederlassung"})
merged_df = merged_df.drop("Niederlassung_x", axis = 1)
merged_df

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNR,Niederlassung
0,64,66894.00,88160.31,1,43.0,K0310,Bayern
1,54,77644.00,103145.70,1,40.0,K1042,Berlin
2,55,44341.00,80565.16,0,37.0,K0382,Baden-Württemberg
3,49,67271.00,83949.89,0,42.0,K0498,Bayern
4,46,49832.00,93781.58,0,41.0,K0552,Hessen
...,...,...,...,...,...,...,...
1990,32,34391.82,NaN,1,48.0,K1211,Hessen
1991,41,68516.50,NaN,0,34.0,K1375,Sachsen
1992,65,89685.33,NaN,0,23.0,K1965,Bayern
1993,68,95714.31,NaN,0,32.0,K1540,Bayern


# 13 Namen der Niederlassungen anzeigen

In [48]:
merged_df["Niederlassung"].unique()

array(['Bayern', 'Berlin', 'Baden-Württemberg', 'Hessen', 'Thüringen',
       'Sachsen', 'Nordrhein-Westfalen', 'BERLIN', 'Niedersachsen',
       'Hamburg', 'Brandenburg', 'Berlin-Mitte', 'Düsseldorf', 'NRW',
       'Berlin-Charlottenburg'], dtype=object)

# 14 Namen der Bundesländer fixen

In [49]:
merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("BERLIN", "Berlin")
merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("Berlin-Mitte", "Berlin")
merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("Berlin-Charlottenburg", "Berlin")
merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("Düsseldorf", "Nordrhein-Westfalen")
merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("NRW", "Nordrhein-Westfalen")
merged_df

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNR,Niederlassung
0,64,66894.00,88160.31,1,43.0,K0310,Bayern
1,54,77644.00,103145.70,1,40.0,K1042,Berlin
2,55,44341.00,80565.16,0,37.0,K0382,Baden-Württemberg
3,49,67271.00,83949.89,0,42.0,K0498,Bayern
4,46,49832.00,93781.58,0,41.0,K0552,Hessen
...,...,...,...,...,...,...,...
1990,32,34391.82,NaN,1,48.0,K1211,Hessen
1991,41,68516.50,NaN,0,34.0,K1375,Sachsen
1992,65,89685.33,NaN,0,23.0,K1965,Bayern
1993,68,95714.31,NaN,0,32.0,K1540,Bayern


In [50]:
merged_df["Niederlassung"].unique()

array(['Bayern', 'Berlin', 'Baden-Württemberg', 'Hessen', 'Thüringen',
       'Sachsen', 'Nordrhein-Westfalen', 'Niedersachsen', 'Hamburg',
       'Brandenburg'], dtype=object)

Kunden stammen aus 10 Bundesländern

# 15.1 Neue Spalte namens "Typ" erstellen (Kunden oder Besucher)

In [51]:
merged_df["Typ"] = merged_df["Preis"].apply(lambda x: 'Kunde' if pd.notna(x) else 'Besucher')

# 15.2 Spalte umpositionieren

In [53]:
merged_df.insert(0, "Typ", merged_df.pop("Typ"))
merged_df

,Typ,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNR,Niederlassung
0,Kunde,64,66894.00,88160.31,1,43.0,K0310,Bayern
1,Kunde,54,77644.00,103145.70,1,40.0,K1042,Berlin
2,Kunde,55,44341.00,80565.16,0,37.0,K0382,Baden-Württemberg
3,Kunde,49,67271.00,83949.89,0,42.0,K0498,Bayern
4,Kunde,46,49832.00,93781.58,0,41.0,K0552,Hessen
...,...,...,...,...,...,...,...,...
1990,Besucher,32,34391.82,NaN,1,48.0,K1211,Hessen
1991,Besucher,41,68516.50,NaN,0,34.0,K1375,Sachsen
1992,Besucher,65,89685.33,NaN,0,23.0,K1965,Bayern
1993,Besucher,68,95714.31,NaN,0,32.0,K1540,Bayern


In [54]:
merged_df["Typ"].value_counts().get("Kunde")

1104

In [55]:
merged_df["Typ"].value_counts().get("Besucher")

891

# 16 CSV Speichern

In [56]:
merged_df.to_csv("data/df_final.csv", index = False)

In [57]:
df_1.to_csv("data/df_1.csv", index = False)

In [58]:
df_2.to_csv("data/df_2.csv", index = False)

In [59]:
df_3.to_csv("data/df_3.csv", index = False)

# 17 Kundennummer als Index

In [60]:
merged_df.set_index("KundeNR", inplace=True)
merged_df

,Typ,Alter,Einkommen,Preis,Geschlecht,Zeit,Niederlassung
KundeNR,,,,,,,
K0310,Kunde,64,66894.00,88160.31,1,43.0,Bayern
K1042,Kunde,54,77644.00,103145.70,1,40.0,Berlin
K0382,Kunde,55,44341.00,80565.16,0,37.0,Baden-Württemberg
K0498,Kunde,49,67271.00,83949.89,0,42.0,Bayern
K0552,Kunde,46,49832.00,93781.58,0,41.0,Hessen
...,...,...,...,...,...,...,...
K1211,Besucher,32,34391.82,NaN,1,48.0,Hessen
K1375,Besucher,41,68516.50,NaN,0,34.0,Sachsen
K1965,Besucher,65,89685.33,NaN,0,23.0,Bayern


# Frage 15: Prognostiziere den Kaufpreis für(m, 32, 30000) & (m, 51, 54000)

In [65]:

df_1.dropna(inplace=True)
X = df_1[["Alter", "Geschlecht", "Einkommen"]]
y = df_1["Preis"]

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Preis   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     1769.
Date:                Fri, 29 Dec 2023   Prob (F-statistic):               0.00
Time:                        11:35:53   Log-Likelihood:                -11749.
No. Observations:                1104   AIC:                         2.351e+04
Df Residuals:                    1100   BIC:                         2.353e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        896.3274   1389.783      0.645      0.519   -1830.598    3623.253
Alter        480.3620     39.499     12.161      0.000     402.860     557.864
Geschlecht  2134.1838    636.779      3.352      0.001     884.744    3383.623
Einkommen      1.0614      0.029     37.095      0.000       1.005       1.118
==============================================================================
Omnibus:                        1.239   Durbin-Watson:                   2.111
Prob(Omnibus):                  0.538   Jarque-Bera (JB):                1.244
Skew:                           0.016   Prob(JB):                        0.537
Kurtosis:                       2.839   Cond. No.                     2.81e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.81e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [63]:
# Ab hier Probleme 

df_test = pd.DataFrame({"Geschlecht": [1, 1],
                        "Alter":      [32, 51], 
                        "Einkommen": [30000, 54000]})

df_test = sm.add_constant(df_test)

predictions = model.predict(df_test)

predictions


ValueError: shapes (2,3) and (4,) not aligned: 3 (dim 1) != 4 (dim 0)